In [25]:
import pandas as pd
import numpy as np
import cv2
import os
import random

In [3]:
''' in this notebook we can preprocess images for the CNN. all the images are super nice and prestine, biut we want them to be distorted 
to an extent or "preprocessed" so the CNN can recognize images pf even low quality'''

# the model takes images of size 224,224,3

' in this notebook we can preprocess images for the CNN. all the images are super nice and prestine, biut we want them to be distorted \nto an extent or "preprocessed" so the CNN can recognize images pf even low quality'

In [5]:
# filepaths 
Test_images = '/Users/melissaaprilcastro/FeatherFind/Data/CUB_200_2011/Test'
Train_images = '/Users/melissaaprilcastro/FeatherFind/Data/CUB_200_2011/Train'

In [29]:
# function to collect all image paths from nested subdirectories
def get_image_paths(folder):
    image_paths = []
    # Traverse the directory and subdirectories
    for root, _, files in os.walk(folder):
        for image in files:
            # Check if the file is an image by its extension
            if image.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff')):
                image_paths.append(os.path.join(root, image))
    return image_paths  # Return the list of image paths

# function to check image dimensions and size of a certain amt of images
def check_random_image_sizes(folder, num_samples=10):
    # Get all image files from nested subdirectories
    image_paths = get_image_paths(folder)
    
    # check if there are enough images to sample (there are lol)
    if len(image_paths) < num_samples:
        print(f"Only found {len(image_paths)} images, processing all of them.")
        selected_images = image_paths
    else:
        # randomly select a subset of images
        selected_images = random.sample(image_paths, num_samples)
    
    # process each selected image
    for file_path in selected_images:
        try:
            # read the image using OpenCV
            img = cv2.imread(file_path)
            
            # check if image was loaded correctly
            if img is not None:
                # get dimensions: height, width, and number of channels
                height, width, channels = img.shape
                
                # get file size in bytes
                file_size = os.path.getsize(file_path)
                
                print(f"Image: {os.path.basename(file_path)}")
                print(f"  - Dimensions: {width} x {height}")
                print(f"  - Channels: {channels}")
                print(f"  - File size: {file_size} bytes\n")
            else:
                print(f"Could not read image: {os.path.basename(file_path)}")
        except Exception as e:
            print(f"Error processing {os.path.basename(file_path)}: {e}")

# run the function to check 10 random images from Train and Test directories
check_random_image_sizes(Train_images, num_samples=10)
check_random_image_sizes(Test_images, num_samples=10)

Image: Henslow_Sparrow_0058_796616.jpg
  - Dimensions: 300 x 380
  - Channels: 3
  - File size: 17798 bytes

Image: Mockingbird_0078_80426.jpg
  - Dimensions: 500 x 375
  - Channels: 3
  - File size: 51724 bytes

Image: Common_Yellowthroat_0035_190567.jpg
  - Dimensions: 500 x 407
  - Channels: 3
  - File size: 95302 bytes

Image: Western_Meadowlark_0120_77834.jpg
  - Dimensions: 500 x 333
  - Channels: 3
  - File size: 71278 bytes

Image: Louisiana_Waterthrush_0058_177388.jpg
  - Dimensions: 500 x 315
  - Channels: 3
  - File size: 129736 bytes

Image: Lincoln_Sparrow_0057_117334.jpg
  - Dimensions: 500 x 334
  - Channels: 3
  - File size: 147061 bytes

Image: Blue_Winged_Warbler_0063_161810.jpg
  - Dimensions: 500 x 383
  - Channels: 3
  - File size: 150253 bytes

Image: Mockingbird_0057_79643.jpg
  - Dimensions: 500 x 379
  - Channels: 3
  - File size: 81786 bytes

Image: Fish_Crow_0010_25836.jpg
  - Dimensions: 500 x 332
  - Channels: 3
  - File size: 65682 bytes

Image: Blue_Grosb

In [36]:
from tensorflow.keras import layers
# we are using efficient net B3 but we can downgrade if it is too computationally expensive
from tensorflow. keras.applications import EfficientNetB3
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [35]:
# base this off the model u use
IMG_SIZE = 300

SIZE = (IMG_SIZE, IMG_SIZE)
# base this off the nu,ber of classification species
NUM_CLASSES = 200

# now we make a validation set too, using image gener
data_gen = 